In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [21]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [24]:
#census_data = pd.read_sql("SELECT * FROM Census_Data", conn)
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [65]:
# View all of the classes that automap found
for Class in Base.classes.keys():
    print(Class)

measurement
station


In [26]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [29]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [125]:
# some classes to display queries
def print_query(query):
    """
    Display all the variables in the item query.
    
        query -- A query, which in this case could be a class instance or a collection of class instances as a sqlalchemy.orm.query.Query.
    """
    if type(query) is sqlalchemy.orm.query.Query:
        for item in query:
            print_query(item)
    else:
        print(f"{'-'*4}class--{query.__class__.__name__}{'-'*4}")
        for key, value in query.__dict__.items():
            if not key.startswith("_"):
                print(f'{key} : {value}')

In [127]:
# Display some starter data
print_query(session.query(Measurement).first())
print_query(session.query(Station).limit(1))

----class--measurement----
tobs : 65.0
date : 2010-01-01
station : USC00519397
id : 1
prcp : 0.08
----class--station----
longitude : -157.8168
name : WAIKIKI 717.2, HI US
station : USC00519397
elevation : 3.0
latitude : 21.2716
id : 1


# Exploratory Precipitation Analysis

In [128]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement).\
    order_by(Measurement.date.desc()).first().date
most_recent_date

'2017-08-23'

In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [11]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [12]:
# Design a query to calculate the total number stations in the dataset


In [13]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [14]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [15]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [66]:
# Close Session
session.close()